In [82]:
import pandas as pd
import numpy as np

1. merging 10 US vital data sets and cleaning

In [83]:
df1 = pd.read_csv("Underlying Cause of Death, 2006.txt",  delimiter="\t")
df2 = pd.read_csv("Underlying Cause of Death, 2007.txt",  delimiter="\t")
df3 = pd.read_csv("Underlying Cause of Death, 2008.txt",  delimiter="\t")
df4 = pd.read_csv("Underlying Cause of Death, 2009.txt",  delimiter="\t")
df5 = pd.read_csv("Underlying Cause of Death, 2010.txt",  delimiter="\t")
df6 = pd.read_csv("Underlying Cause of Death, 2011.txt",  delimiter="\t")
df7 = pd.read_csv("Underlying Cause of Death, 2012.txt",  delimiter="\t")
df8 = pd.read_csv("Underlying Cause of Death, 2013.txt",  delimiter="\t")
df9 = pd.read_csv("Underlying Cause of Death, 2014.txt",  delimiter="\t")
df10 = pd.read_csv("Underlying Cause of Death, 2015.txt",  delimiter="\t")

In [84]:
# merged 10 data sets

df=[df1,df2,df3, df4, df5, df6, df7, df8, df9, df10]
df_new=pd.concat(df)

In [85]:
df_new.head()

,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths
0,NaN,"Autauga County, AL",1001.0,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,449
1,NaN,"Baldwin County, AL",1003.0,2006.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,D1,11
2,NaN,"Baldwin County, AL",1003.0,2006.0,2006.0,All other alcohol-induced causes,A9,15
3,NaN,"Baldwin County, AL",1003.0,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,1613
4,NaN,"Barbour County, AL",1005.0,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,284


In [86]:
#split state and county name

temp=df_new['County'].str.split(",",expand=True)
df_new['County']=temp[0]
df_new['State']=temp[1]

In [87]:
df_new['Drug/Alcohol Induced Cause'].unique()

array(['All other non-drug and non-alcohol causes',
       'Drug poisonings (overdose) Unintentional (X40-X44)',
       'All other alcohol-induced causes',
       'All other drug-induced causes',
       'Drug poisonings (overdose) Undetermined (Y10-Y14)',
       'Drug poisonings (overdose) Suicide (X60-X64)', nan,
       'Alcohol poisonings (overdose) (X45, X65, Y15)',
       'Drug poisonings (overdose) Homicide (X85)'], dtype=object)

In [88]:
#select 'Drug poisonings (overdose) Unintentional (X40-X44)', 'Drug poisonings (overdose) Undetermined (Y10-Y14)', 'Drug poisonings (overdose) Suicide (X60-X64)'
df_death=df_new.loc[df_new['Drug/Alcohol Induced Cause'].isin(['Drug poisonings (overdose) Unintentional (X40-X44)', 'Drug poisonings (overdose) Undetermined (Y10-Y14)', 'Drug poisonings (overdose) Suicide (X60-X64)'])]

In [89]:
#reorder col names
columnsTitles = ['Notes', 'State', 'County', 'County Code', 'Year', 'Year Code',
       'Drug/Alcohol Induced Cause', 'Drug/Alcohol Induced Cause Code',
       'Deaths']

df_death = df_death.reindex(columns=columnsTitles).copy()

In [90]:
#drop useless cols

df_death=df_death.drop(["Notes", "Year Code", "Drug/Alcohol Induced Cause Code"],axis=1 )

In [91]:
df_death.head()

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths
1,AL,Baldwin County,1003.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,11
12,AL,Chilton County,1021.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,13
39,AL,Jefferson County,1073.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,55
55,AL,Mobile County,1097.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,23
56,AL,Mobile County,1097.0,2006.0,Drug poisonings (overdose) Undetermined (Y10-Y14),10


In [92]:
df_death

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths
1,AL,Baldwin County,1003.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,11
12,AL,Chilton County,1021.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,13
39,AL,Jefferson County,1073.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,55
55,AL,Mobile County,1097.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,23
56,AL,Mobile County,1097.0,2006.0,Drug poisonings (overdose) Undetermined (Y10-Y14),10
...,...,...,...,...,...,...
4744,WI,Waukesha County,55133.0,2015.0,Drug poisonings (overdose) Unintentional (X40-...,34
4750,WI,Winnebago County,55139.0,2015.0,Drug poisonings (overdose) Unintentional (X40-...,22
4760,WY,Fremont County,56013.0,2015.0,Drug poisonings (overdose) Unintentional (X40-...,10
4766,WY,Laramie County,56021.0,2015.0,Drug poisonings (overdose) Unintentional (X40-...,13


In [93]:
#cleaning missing value
df_death['State']=df_death['State'].astype(str)
df_death['County']=df_death['County'].astype(str)
df_death['Deaths']=df_death['Deaths'].astype(str)
df_death= df_death[~df_death['Deaths'].str.contains("Missing")]



In [94]:
#change data type

df_death['County Code']=df_death['County Code'].astype(int)
df_death['Year']=df_death['Year'].astype(int)


C:\Users\roder\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\roder\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [95]:
df_death['Deaths']=pd.to_numeric(df_death['Deaths'], downcast='integer')

C:\Users\roder\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
df_death.head()

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths
1,AL,Baldwin County,1003,2006,Drug poisonings (overdose) Unintentional (X40-...,11
12,AL,Chilton County,1021,2006,Drug poisonings (overdose) Unintentional (X40-...,13
39,AL,Jefferson County,1073,2006,Drug poisonings (overdose) Unintentional (X40-...,55
55,AL,Mobile County,1097,2006,Drug poisonings (overdose) Unintentional (X40-...,23
56,AL,Mobile County,1097,2006,Drug poisonings (overdose) Undetermined (Y10-Y14),10


2. population data cleaning

In [97]:
df2= pd.read_excel("PopulationReport.xlsx")

In [98]:
df2.head()


,State,County Name,pop2010
0,AL,Alabama,4780138
1,AL,Autauga County,54574
2,AL,Baldwin County,182264
3,AL,Barbour County,27457
4,AL,Bibb County,22920


In [99]:
df2.rename(columns = {'County Name':'County'}, inplace = True)
df2['State']=df2['State'].astype(str)
df2['County']=df2['County'].astype(str)
df2['pop2010']=df2['pop2010'].astype(int)

3. Final data set

In [123]:
#merging df_death and df2 to get final data set

df_final=pd.merge(df_death, df2, how=inner, on=['State', 'County'])

In [124]:
df_final

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths,pop2010


In [26]:
#Death rate Calculation

df_final['Death Rate']=df_final['Deaths']/df_final['pop2010']

In [27]:
df_final.head()

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths,pop2010,Death Rate
